## demo mcmc

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as model

%matplotlib inline

## define functions and constants

In [2]:
def newtheta(theta):
    a = 0.05
    fa = lambda x:x + a*x*np.random.randn()
    current = theta
    #newkc = mapkc(current.KC.copy())
    #newsig = mapkc(current.sig.copy())
    proposed = current.map(fa)
    #proposed = current.drop(['KC','sig']).map(fa)
    #proposed['KC']=newkc
    #proposed['sigma']=newsig
    return proposed

In [3]:
def chisq(data,model,variance):
    d = data
    m = model
    v = variance

    #for first epoch probability
    fmax = lambda x:(max(x,0.001)**2)
    N1 = m['COUNTS_U1']
    x1 = (m['FIRST_EPOCH_PROB_U1']-d['FIRST_EPOCH_PROB_U1'])**2
    s1 = m['FIRST_EPOCH_PROB_U1'].map(fmax)
    N2 = m['COUNTS_U2']
    x2 = (m['FIRST_EPOCH_PROB_U2']-d['FIRST_EPOCH_PROB_U2'])**2
    s2 = m['FIRST_EPOCH_PROB_U2'].map(fmax)

    #other chi's follow form: chi = ((d-m)**2)/(v**2)
    chi = pd.DataFrame()
    chi['FIRST_EPOCH_PROB_U1'] = (N1*(x1/s1) + N2*(x2/s2))
    chi['MEAN_U1'] = ((m['MEAN_U1']-d['MEAN_U1'])**2)/(v['MEAN_U1']**2)
    chi['MEAN_U2'] = ((m['MEAN_U2']-d['MEAN_U2'])**2)/(v['MEAN_U2']**2)
    chi['SD_U1'] = ((m['SD_U1']-d['SD_U1'])**2)/(v['SD_U1']**2)
    chi['SD_U2'] = ((m['SD_U2']-d['SD_U2'])**2)/(v['SD_U2']**2)

    return chi

In [4]:
def chisum(df):
    #return scalar
    one = df.sum(axis=0)
    two = one.sum(0)
    return two

In [5]:
def taupen(tau_a):
    penalty = (tau_a-2)**2
    return penalty

def kpen(ks,kc):
    delta = 0
    pen = delta*(ks + kc)**2
    return pen

In [23]:
index = [0,20,40,60,80]
cv = 0.6
cv_var = 0.2
p1_var = 0.1
fcv = lambda x: x*cv
fcv_var = lambda x:(x*cv_var)
fp1_var = lambda x:(x*p1_var)

## load data

In [7]:
syprob = pd.read_csv('sy_prob.csv',index_col=0)
syU1 = pd.read_csv('syU1.csv',index_col=0)
syU2 = pd.read_csv('syU2.csv',index_col=0)

## construct dataframe of data

In [8]:
data = pd.DataFrame()
data['COUNTS_U1'] = syprob['counts_U1']
data['COUNTS_U2'] = syprob['counts_U2']
data['FIRST_EPOCH_PROB_U1'] = syprob['U1']
data['FIRST_EPOCH_PROB_U2'] = syprob['U2']
data['MEAN_U1'] = syU1['MEAN']
data['MEAN_U2'] = syU2['MEAN']
data['SD_U1'] = data['MEAN_U1'].map(fcv)
data['SD_U2'] = data['MEAN_U2'].map(fcv)
data.index=index
col = data.columns

In [9]:
data

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,50,50,0.50,0.50,1800.0,1700.0,1080.0,1020.0
20,95,5,0.95,0.05,1900.0,1700.0,1140.0,1020.0
40,99,1,0.99,0.01,2200.0,1700.0,1320.0,1020.0
60,100,0,1.00,0.00,2700.0,1700.0,1620.0,1020.0
80,100,0,1.00,0.00,3200.0,1700.0,1920.0,1020.0


## calculate error and make dataframe from data

In [10]:
variance = pd.DataFrame()
variance['COUNTS_U1'] = data['COUNTS_U1'].map(fp1_var)
variance['COUNTS_U2'] = data['COUNTS_U2'].map(fp1_var)
variance['FIRST_EPOCH_PROB_U1'] = data['FIRST_EPOCH_PROB_U1'].map(fp1_var)
variance['FIRST_EPOCH_PROB_U2'] = data['FIRST_EPOCH_PROB_U2'].map(fp1_var)
variance['MEAN_U1'] = data['MEAN_U1'].map(fcv_var)
variance['MEAN_U2'] = data['MEAN_U2'].map(fcv_var)
variance['SD_U1'] = data['SD_U1'].map(fcv_var)
variance['SD_U2'] = data['SD_U2'].map(fcv_var)
variance.index = index
variance.columns = col

In [11]:
variance

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,5.0,5.0,0.050,0.050,450.0,425.0,270.0,255.0
20,9.5,0.5,0.095,0.005,475.0,425.0,285.0,255.0
40,9.9,0.1,0.099,0.001,550.0,425.0,330.0,255.0
60,10.0,0.0,0.100,0.000,675.0,425.0,405.0,255.0
80,10.0,0.0,0.100,0.000,800.0,425.0,480.0,255.0


### load initial parameters

In [12]:
!ls

README.md            couperin.pkl         modelequations2.0.py
Rivalry.jl           demodynamics.ipynb   rivalry.py
bach.pkl             demomcmc.ipynb       rivalry.pyc
barber.pkl           demomodel.ipynb      storeresults
beethoven.pkl        initials             syU1.csv
bizet.pkl            mcmc.py              syU2.csv
brahms.pkl           mcmc.pyc             sy_bias.csv
bruch.pkl            model.py             sy_prob.csv
chopin.pkl           model.pyc            temp


In [24]:
initial = pd.read_pickle('brahms.pkl')
initial

alpha       0.000114
beta        0.938066
gamma       1.053626
kb          0.128695
kc          0.005298
ks          0.206230
mu          1.187046
sigma       0.030758
tau_a    1694.962846
tau_u       8.100646
Name: 26, dtype: float64

In [25]:
theta_cur = initial
theta_prop = newtheta(theta_cur)

In [26]:
print theta_cur
print theta_prop

alpha       0.000114
beta        0.938066
gamma       1.053626
kb          0.128695
kc          0.005298
ks          0.206230
mu          1.187046
sigma       0.030758
tau_a    1694.962846
tau_u       8.100646
Name: 26, dtype: float64
alpha       0.000115
beta        0.990621
gamma       1.162190
kb          0.130449
kc          0.005275
ks          0.219655
mu          1.254226
sigma       0.032759
tau_a    1848.660894
tau_u       7.642105
Name: 26, dtype: float64


## run model with current and proposed theta values

In [27]:
#trials
dominance = 1
firstprob = 200

In [28]:
m1 = model.run(theta_cur,dominance,firstprob)

In [46]:
#m2 = model.run(theta_prop,dominance,firstprob)

In [29]:
m1.index = index
m1.columns = col
m1

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,92.0,108.0,0.460,0.540,1079.475162,1069.030303,625.833270,620.896265
20,167.0,33.0,0.835,0.165,1233.102450,982.685969,699.040525,549.416481
40,198.0,2.0,0.990,0.010,1387.267574,869.092971,836.230830,517.920828
60,199.0,1.0,0.995,0.005,1746.169231,802.835897,1018.101203,458.835230
80,200.0,0.0,1.000,0.000,2343.560748,735.352025,1493.593005,430.820283


In [32]:
m1['CV U1'] = m1['SD_U1']/m1['MEAN_U1']
m1['CV U2'] = m1['SD_U2']/m1['MEAN_U2']
m1

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2,CV U1,CV U2
0,92.0,108.0,0.460,0.540,1079.475162,1069.030303,625.833270,620.896265,0.579757,0.580803
20,167.0,33.0,0.835,0.165,1233.102450,982.685969,699.040525,549.416481,0.566896,0.559097
40,198.0,2.0,0.990,0.010,1387.267574,869.092971,836.230830,517.920828,0.602790,0.595933
60,199.0,1.0,0.995,0.005,1746.169231,802.835897,1018.101203,458.835230,0.583048,0.571518
80,200.0,0.0,1.000,0.000,2343.560748,735.352025,1493.593005,430.820283,0.637318,0.585869


In [45]:
#m2.index = index
#m2.columns=col
#m2

,COUNTS_U1,COUNTS_U2,FIRST_EPOCH_PROB_U1,FIRST_EPOCH_PROB_U2,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,96.0,104.0,0.480,0.520,1521.141975,1512.870370,520.446317,519.955069
20,121.0,79.0,0.605,0.395,1593.728395,1459.968944,541.525087,494.729105
40,127.0,73.0,0.635,0.365,1714.386076,1393.525316,561.673536,445.849618
60,140.0,60.0,0.700,0.300,1833.316129,1343.597403,580.043666,421.527995
80,151.0,49.0,0.755,0.245,1993.738255,1327.255034,572.321722,368.801999


## caluclate chi construct dataframe

In [30]:
chicur = pd.DataFrame(chisq(data,m1,variance),index=index)
#chiprop = pd.DataFrame(chisq(data,m2,variance),index=index)

In [31]:
chicur

,FIRST_EPOCH_PROB_U1,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,1.288245,2.563734,2.204140,2.829457,2.449578
20,19.197968,1.971202,2.848661,2.393909,3.405596
40,0.000000,2.183583,3.822320,2.149060,3.876717
60,1.005025,1.996802,4.456213,2.208701,4.842844
80,0.000000,1.146075,5.151810,0.789162,5.338450


In [53]:
chiprop

,FIRST_EPOCH_PROB_U1,MEAN_U1,MEAN_U2,SD_U1,SD_U2
0,0.320513,0.600014,0.302920,6.710827,6.008385
20,99.612930,0.649600,0.498399,6.890047,6.629890
40,108.747708,1.218083,0.812515,8.250962,7.921200
60,85.714286,2.575929,1.098813,10.302442,8.606515
80,64.900662,3.552411,1.201893,12.317144,10.189803


##  calulate likelihood ratio

In [54]:
sumchicur = chisum(chicur)
sumprop = chisum(chiprop)
print sumchicur
print sumprop

370.087457924
455.633890042


In [55]:
ratio = np.exp((-sumprop+sumchicur)/2)
ratio

2.6535562151025129e-19